In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

In [ ]:
kh = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
kh.head()

zz = kh.copy()#Pour utilizer ce dataframe dans la partie zipcodes
kh[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','price','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ].describe().astype(int)

In [ ]:
kh['price'].hist(bins=64)


In [ ]:
kh.loc[kh['sqft_living15']<250000 ,'sqft_living15'].hist(bins=100)

In [ ]:
kh = kh[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','price','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ]


#l'utilization de logarithme dans les colomn qui n'ont pas un moyen element

kh['log_sqft_lot']=np.log(1+kh['sqft_lot'])
kh['log_sqft_living']=np.log(1+kh['sqft_living'])
kh['log_sqft_living15']=np.log(1+kh['sqft_living15'])
kh['log_sqft_lot15']=np.log(1+kh['sqft_lot15'])
kh['log_sqft_basement']=np.log(1+kh['sqft_basement'])
kh['log_sqft_above']=np.log(1+kh['sqft_above'])
kh['log_bedrooms']=np.log(1+kh['bedrooms'])
kh['log_price']=np.log(1+kh['price'])
kh[['log_sqft_lot','log_sqft_living','floors','log_price','log_sqft_living15','log_sqft_lot15','log_sqft_basement','log_sqft_above']].describe().astype(int)

train_kh = kh[0:17000] #80% des données apprentissage
test_kh = kh[17000:] #20% des données pour le test

#df['log_sqft_lot'].hist(bins=100)




In [ ]:
kh['log_sqft_living'].hist(bins=100)

In [ ]:
kh[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','price','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ].describe().astype(int)

In [ ]:
#Separer les données en X et Y
train_A = train_kh[ ['log_sqft_lot','log_sqft_living','floors','log_sqft_above','log_sqft_basement','log_sqft_lot15','log_sqft_living15','bathrooms','log_bedrooms','grade'] ].values
train_B = train_kh[ 'log_price' ].values

test_A= test_kh[ ['log_sqft_lot','log_sqft_living','floors','log_sqft_above','log_sqft_basement','log_sqft_lot15','log_sqft_living15','bathrooms','log_bedrooms','grade'] ].values
test_B = test_kh[ 'log_price' ].values

print( train_A.shape, test_A.shape )
print( train_B.shape, test_B.shape )

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras.models import Model
from tensorflow.keras.optimizers import SGD



In [ ]:
entre = layers.Input( shape=(10,) ,name='entree' )
a    =  layers.Dense(500) ( entre )
actv =  layers.Activation('relu') (a)
sortie = layers.Dense(1) (actv)

model = Model( entre, sortie )

model.summary()

In [ ]:
opt = SGD() 

model.compile( optimizer=opt, loss='mse' )

In [ ]:
model.fit( train_A, train_B, epochs=10 )


In [ ]:
kh['price']

In [ ]:
prediction = model.predict(test_A)
prediction
np.exp(prediction)-1

In [ ]:
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler


In [ ]:
kh_maxmin = kh[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ]#pour utiliser dans la normalization min max

# la normalisation min max using from the library sklearn 

scaler = MinMaxScaler ()

scaler.fit(kh_maxmin)

A_scaled = scaler.transform (kh_maxmin)

A_scaled.shape

In [ ]:
##price nomralization using maxmin same library

Ascaler = MinMaxScaler ()

prix = kh[['price']]

Ascaler.fit(prix)

B_scaled = Ascaler.transform (prix)


B_scaled

In [ ]:
#we use again the neural network to check our new prediction values using this new normalized data 


entre = layers.Input( shape=(10,) ,name='entree' )
a   =  layers.Dense(700) ( entre )
actv =  layers.Activation('relu') (a)
sortie = layers.Dense(1) (actv)

modelmaxmin = Model( entre, sortie )

modelmaxmin.summary()

In [ ]:
# la normalisation min max method 2
# train_X=train_X.apply(lambda x : (x-x.min(axis=0))/(x-x.max(axis=0)-x.min(axis=0)) )


In [ ]:
##using the same algorithme Sgd in all forms to check which algorithm works better

optminmax = SGD() 




In [ ]:
modelmaxmin.compile( optimizer=optminmax, loss='mse' )

In [ ]:
modelmaxmin.fit( A_scaled[0:17000], B_scaled[0:17000], epochs=10 )

In [ ]:
B_scaled[17000:]

In [ ]:
predictions = modelmaxmin.predict(X_scaled[17000:])
predictions = Ascaler.inverse_transform(predictions)
predictions

In [ ]:
kh_Z_Score = kh[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ]#pour utiliser dans la normalization ecart type

##using the Z-score normalization which is called standard scaler from the library sklearn
##using the data from the original data frame 
Z_Score_scaler = StandardScaler()
# transform data
Z_scaled = Z_Score_scaler.fit_transform(kh_Z_Score)

Z_scaled



In [ ]:
Z_score_price = kh[['price']]


## using Z-score normalization in the data frame of price as well so we have same range for all the data
Z_price_scaler = StandardScaler()
# transform data
Z_priced = Z_price_scaler.fit_transform(Z_score_price)

Z_priced



In [ ]:
##Trying the neural network again with a differen approach this time using normalized data by Z-score

entre = layers.Input( shape=(10,) ,name='entree' )
a    =  layers.Dense(700) ( entre )
actv =  layers.Activation('relu') (a)
sortie = layers.Dense(1) (actv)

model_Z_Score = Model( entre, sortie )

model_Z_Score.summary()

In [ ]:
opt_Zscore = SGD() 

model_Z_Score.compile( optimizer=opt_Zscore, loss='mse' )

model_Z_Score.fit( Z_scaled[0:17000], Z_priced[0:17000], epochs=10 )

In [ ]:
prix[:6]

In [ ]:
predictionsZ = model_Z_Score.predict(Z_scaled[17000:])
predictionsZ = Z_price_scaler.inverse_transform(predictionsZ)
predictionsZ[:6]

In [ ]:
zz[['zipcode']]

In [ ]:
zip_dums = pd.get_dummies(zz['zipcode'])

df_Zipcodes = pd.DataFrame(X_scaled)##Converting the np array to a dataframe

 

#Concatination between min max scaling and one hot encoded zipcodes
zip_df = pd.concat([ df_Zipcodes, zip_dums],axis=1)

zip_df

In [ ]:
entre = layers.Input( shape=(80,) ,name='entree' )
a    =  layers.Dense(500) ( entre )
actv =  layers.Activation('relu') (a)
sortie = layers.Dense(1) (actv)

modelzipcodes = Model( entre, sortie )

modelzipcodes.summary()

In [ ]:
opt_zipcodes = SGD() 

modelzipcodes.compile( optimizer=opt_zipcodes, loss='mse' )

modelzipcodes.fit( zip_df[0:17000], B_scaled[0:17000], epochs=10 )

In [ ]:
predictions_Zipcodes = modelzipcodes.predict(zip_df[17000:])
predictionsEndMaxMin = Ascaler.inverse_transform(predictions_Zipcodes)
predictionsEndMaxMin

In [ ]:
df_Zipcodes_Zscore = pd.DataFrame(Z_scaled)

zip_df_Zscore = pd.concat([ df_Zipcodes_Zscore, zip_dums],axis=1)

zip_df_Zscore

In [ ]:
entre = layers.Input( shape=(80,) ,name='entree' )
a    =  layers.Dense(500) ( entre )
actv =  layers.Activation('relu') (a)
sortie = layers.Dense(1) (actv)

modelzipcodes_Zscore = Model( entre, sortie )

modelzipcodes_Zscore.summary()

In [ ]:
opt_zipcodes_Zscore = SGD() 

modelzipcodes_Zscore.compile( optimizer=opt_zipcodes_Zscore, loss='mse' )

modelzipcodes_Zscore.fit( zip_df_Zscore[0:17000], Z_priced[0:17000], epochs=10 )

In [ ]:
predictions_Zipcodes_Zscore = modelzipcodes_Zscore.predict(zip_df_Zscore[17000:])
predictions_End_Zscore = Z_price_scaler.inverse_transform(predictions_Zipcodes_Zscore)
predictions_End_Zscore